# Generate docs for validation rules taxo

## Read the XBRL-instance

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst
from arelle import RenderingEvaluator

In [ ]:
import src
import pandas as pd
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from io import StringIO
import ast

In [ ]:
# make sure you have a 'arelle' directory in the data_path! (This is where the taxonomy is stored)
XBRL_DATA_PATH    = '..\\data\\taxonomy\\' 
XBRL_RESULTS_PATH = '..\\data\\' 
LANGUAGE     = "en-GB"
# set the location of taxonomy
environ['XDG_CONFIG_HOME'] = XBRL_DATA_PATH

In [ ]:
DOCS_PATH ="..\\docs\\"
DATA_PATH ="..\\data\\"
RULES_PATH = "..\\Solvency2-rules\\"
DIR_NAME = "EIOPA Validation rules_2.4.0\\"

In [ ]:
# Now we make a modelmanager
controller = Cntlr.Cntlr()
controller.webCache.workOffline = True

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [ ]:
if (not exists(join(DATA_PATH, "datapoints.txt"))) and (not exists(join(DATA_PATH, "templates.txt"))):
    # Constructing dictionary of datapoints in rc-format
    # This will take a while
    template_dict = {}
    datapoint_dict = {}
    instances = [file for file in listdir(XBRL_DATA_PATH) if file.endswith(".xbrl")]
    # And we read the XBRL instances to extract datapoints in rc-format
    for file in instances:
        print(file)
        xbrl_instance = ModelXbrl.load(modelmanager, join(XBRL_DATA_PATH, file))
        linkRoleUris = xbrl_instance.relationshipSet("Table-rendering").linkRoleUris
        for role_uri in linkRoleUris:
            definition = basename(role_uri)
            tblAxisRelSet = xbrl_instance.relationshipSet(XbrlConst.euTableAxis, role_uri)
            if len(tblAxisRelSet.modelRelationships)==0:
                tblAxisRelSet = xbrl_instance.relationshipSet((XbrlConst.tableBreakdown, XbrlConst.tableBreakdownMMDD, XbrlConst.tableBreakdown201305, XbrlConst.tableBreakdown201301, XbrlConst.tableAxis2011), role_uri)
            for rootconcept in tblAxisRelSet.rootConcepts:
                # rootconcept is a modelTable-object
                template_dict[rootconcept.definitionLabelsView[2][1]] = rootconcept.definitionLabelsView[3][1]
        tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
        RenderingEvaluator.init(xbrl_instance)
        datapoint_dict.update(src.rc2label.rc2label_dict(xbrl_instance))
        
    f = open(join(DATA_PATH, "datapoints.txt"),"w")
    f.write(str(datapoint_dict))
    f.close()
    f = open(join(DATA_PATH, "templates.txt"),"w")
    f.write(str(template_dict))
    f.close()
else:
    f = open(join(DATA_PATH, "datapoints.txt"),"r")
    datapoint_dict = ast.literal_eval(f.read())
    f.close()
    f = open(join(DATA_PATH, "templates.txt"),"r")
    template_dict = ast.literal_eval(f.read())
    f.close()

## List validation rules in taxonomy

In [ ]:
XBRL_INSTANCE = 'examples\\qrs_240_instance.xbrl'

xbrl_instance = ModelXbrl.load(modelmanager, XBRL_DATA_PATH + XBRL_INSTANCE)

ViewFileFormulae.viewFormulae(xbrl_instance, XBRL_RESULTS_PATH + "formulae.csv", "header", None)
formulae = pd.read_csv(XBRL_RESULTS_PATH + "formulae.csv")
df_xbrl = formulae[formulae['Expression'].str[0:2]=="BV"]

In [ ]:
len(df_xbrl.index)

In [ ]:
df = pd.DataFrame()
for row in df_xbrl.index:
    expr = df_xbrl.loc[row, 'Expression']
    label = df_xbrl.loc[row, 'Label']
    rule_templates, rule_datapoints, rule_id, rule_ref = src.parse_formula(expr, syntax = "XBRL")
    rule_date = "Unknown"
    df = df.append(pd.DataFrame(data = [[rule_id, label, rule_templates, rule_datapoints, rule_ref, expr]]), ignore_index = True)
df.columns = ['Rule id', 'Rule label', 'Rule templates', 'Rule datapoints', 'Rule references', 'Rule expression']

In [ ]:
df.head(5)

## Generate subdirectories for XBRL Formulae

In [ ]:
all_templates = []
for row in df.index:
    all_templates.extend(df.loc[row, "Rule templates"])
templates = list(pd.Series(data = all_templates).sort_values().unique())

for template in templates:
    if not exists(join(DOCS_PATH, DIR_NAME, template)):
        makedirs(join(DOCS_PATH, DIR_NAME, template))

In [ ]:
src.write_rst(join(DOCS_PATH, DIR_NAME), df, template_dict, datapoint_dict)